# Neural Network from scratch

neural network has input neuron, hidden neurons and output neurons.

each and every neuron is connected to all the neurons in its previous layer. and these connections have weights some are strong some are weak. and forming a network

So looking at this out basic building blocks will be

1. Connection -> will keep information between the connection of two neuron

2. Neuron -> will get signals from connected neurons and produce an output

3. Network -> will create a network of the neurons and flow data in the layers

Importing Libraries

In [1]:
import math
import numpy as np

## Designing Connection class

The `Connection` class defines connections between neurons in a neural network. Each connection has a weight and stores the change in weight during training.

In [2]:
class Connection:
    def __init__(self, connected_neuron):
        self.connected_neuron = connected_neuron
        self.weight = np.random.normal()
        # below is the delta weight
        self.delta_weight = 0.0

## Designing a Neuron class 

The `Neuron` class represents a single neuron in a neural network. Key features include:

- **Initialization**: Neurons are initialized with properties such as dendrons (connections to other neurons), error, and gradient.

- **Activation Functions**: It includes functions for the sigmoid activation function and its derivative.

- **Forward Propagation**: `feedForward` calculates the neuron's output based on incoming signals from connected neurons.

- **Backpropagation**: `backPropagate` adjusts weights based on error and propagates error back to connected neurons.

In [3]:
class Neuron:
    # learning rate
    eta = 0.001
    # momentum factor
    alpha = 0.01

    def __init__(self, layer) :
        self.dendrons = []
        self.error = 0.0
        self.gradient = 0.0
        self.output = 0.0
        if layer is None:
            pass
        else:
            for neuron in layer:
                con = Connection(neuron)
                self.dendrons.append(con)

    def addError(self, err):
        self.error += err

    def sigmoid(self, x):
        return 1/ (1 + math.exp(-x * 1.0))
    
    def sigmoidDerivative(self, x):
        return x * (1.0 - x)
    
    def setError(self, err):
        self.error = err

    def setOutput(self, output):
        self.output = output

    def getOutput(self):
        return self.output

    def feedForward(self, inputs):
        sum = 0
        for dendron in self.dendrons:
            sum += dendron.connected_neuron.getOutput() * dendron.weight
        self.output = self.sigmoid(sum)

    def backPropagate(self):
        self.gradient = self.error * self.sigmoidDerivative(self.output)

        for dendron in self.dendrons:
            dendron.delta_weight = self.eta * dendron.connected_neuron.output * self.gradient + self.alpha * dendron.delta_weight

            dendron.weight += dendron.delta_weight

            dendron.connected_neuron.addError(dendron.weight * self.gradient)

        self.error = 0

## Network class

The `Network` class represents a neural network structure with methods for:

- **Initialization**: Initializes the network based on the given topology, creating layers and neurons accordingly.

- **Input Setting**: `setInput` sets the input values for the input layer neurons.

- **Error Calculation**: `getError` computes the root mean square error between network outputs and target values.

- **Forward Propagation**: `feedForward` propagates input signals through the network.

- **Backpropagation**: `backPropagate` updates weights based on errors and propagates errors backward through the network.

- **Result Retrieval**: `getResults` retrieves the output values from the output layer.

- **Thresholded Result Retrieval**: `getThresholdResults` retrieves binary results based on a threshold (0.5) for classification tasks, excluding the bias neuron.

In [4]:
class Network:
    def __init__(self, topology) :
        self.layers = []
        for numNeuron in topology:
            layer = []
            for i in range(numNeuron):
                if (len(self.layers) == 0):
                    layer.append(Neuron(None))
                else:
                    layer.append(Neuron(self.layers[-1]))

                layer.append(Neuron(None))
                layer[-1].setOutput(1)
                self.layers.append(layer)

    def setInput(self, inputs):
        for i in range(len(inputs)):
            self.layers[0][i].setOutput(inputs[i])

    def getError(self, target):
        # here err is taken as the root mean square of the samples 
        err = 0
        for i in range(len(target)):
            e = (target[i] - self.layers[-1][i].getOutput())
            err += e ** 2
        
        err = err/len(target)
        err = math.sqrt(err)
        return err
    
    def feedForward(self, inputs):
        for i in range(len(inputs)):
            self.layers[0][i].setOutput(inputs[i])

        for layer in self.layers[1:]:
            for neuron in layer:
                neuron.feedForward(inputs)

    def backPropagate(self, target):
        for i in range(len(target)):
            self.layers[-1][i].setError(target[i] - self.layers[-1][i].getOutput())

        for layer in self.layers[::-1]:
            for neuron in layer:
                neuron.backPropagate()

    def getResults(self):
        output = []
        for neuron in self.layers[-1]:
            output.append(neuron.getOutput())
        output.pop()
        return output
    
    def getThresholdResults(self):
        output = []
        for neuron in self.layers[-1]:
            result = neuron.getOutput()
            if result > 0.5:
                output.append(1)
            else:
                output.append(0)
        # removing the bias neuron
        output.pop()
        return output


    

    
    

            

## Driver code:

The `main()` function trains and tests a neural network:

- **Network Configuration**: Sets up a neural network with specified input, hidden, and output layer sizes.
- **Training Data**: Defines input-output pairs for training.
- **Training Loop**: Iterates through training data, adjusting weights via backpropagation to minimize error.
- **Testing**: Tests the trained network on a set of test inputs, printing the network's output.

It's a straightforward demonstration of training a neural network to perform logic gate operations (XOR) and then testing it on various input combinations.

In [5]:
def main():
    input_size = 2
    hidden_size = 3
    output_size = 1
    network = Network([input_size, hidden_size, output_size])

    # learning factor:
    Neuron.eta = 0.07

    # Momentum factor :
    Neuron.alpha = 0.01

    # Define training data
    inps = [[0, 0], [0, 1], [1, 0], [1, 1]]
    outs = [[0], [1], [1], [0]]

    # Train the network
    for i in range(10000):
        error = 0
        for inp, out in zip(inps, outs):  
            network.setInput(inp)
            network.feedForward(inp)
            network.backPropagate(out)
            error += network.getError(out)
        print("Epoch %d error: %f" % (i, error))
        if error < 0.01:
            break

    # Test the network
    # Generate test input values
    test_inputs = []
    for i in range(200):  
        for j in range(300):  
            test_inputs.append([i, j])

    for input_values in test_inputs:
        network.setInput(input_values)
        network.feedForward(input_values)
        print("Input:", input_values, "Output:", network.getThresholdResults())


if __name__ == "__main__":
    main()

Epoch 0 error: 2.000699
Epoch 1 error: 2.000716
Epoch 2 error: 2.000734
Epoch 3 error: 2.000752
Epoch 4 error: 2.000771
Epoch 5 error: 2.000790
Epoch 6 error: 2.000810
Epoch 7 error: 2.000830
Epoch 8 error: 2.000851
Epoch 9 error: 2.000873
Epoch 10 error: 2.000896
Epoch 11 error: 2.000919
Epoch 12 error: 2.000943
Epoch 13 error: 2.000967
Epoch 14 error: 2.000993
Epoch 15 error: 2.001019
Epoch 16 error: 2.001046
Epoch 17 error: 2.001074
Epoch 18 error: 2.001102
Epoch 19 error: 2.001132
Epoch 20 error: 2.001162
Epoch 21 error: 2.001194
Epoch 22 error: 2.001226
Epoch 23 error: 2.001260
Epoch 24 error: 2.001294
Epoch 25 error: 2.001330
Epoch 26 error: 2.001366
Epoch 27 error: 2.001404
Epoch 28 error: 2.001443
Epoch 29 error: 2.001483
Epoch 30 error: 2.001524
Epoch 31 error: 2.001566
Epoch 32 error: 2.001610
Epoch 33 error: 2.001654
Epoch 34 error: 2.001701
Epoch 35 error: 2.001748
Epoch 36 error: 2.001796
Epoch 37 error: 2.001846
Epoch 38 error: 2.001898
Epoch 39 error: 2.001950
Epoch 40 e